# Autorec as recommendation engine

Companion notebook for a conference speech about using autoencoders as recommendation engines. It presents an idea of using Autoencoder neural network as a recommendation engine for [famous MovieLens dataset](https://www.kaggle.com/grouplens/movielens-20m-dataset).

# Technical prep

In [1]:
import os
import sys


In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Bar, Scatter, Figure, Layout
from tqdm import tqdm_notebook as tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.sparse import lil_matrix, csr_matrix, save_npz, load_npz

C:\Users\fwojcik\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
%matplotlib inline  
%matplotlib notebook

In [5]:
init_notebook_mode(connected=True)

In [6]:
tqdm().pandas()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

In [9]:
output_notebook()

Loading BokehJS ...

In [10]:
def plot_loss(history, loss_to_plot):
    p = figure(plot_width = 600, plot_height = 400) 

    # add a line renderer 
    p.line(range(len(history.history[loss_to_plot])), history.history[loss_to_plot],  
            line_width = 2, color = "green") 

    p.add_tools(HoverTool(tooltips=[('err', '@y')]))

    # show the results 
    show(p) 

# Data prep

Building a recommendation engine from a **movielens** dataset can be a challenging task. There's a lot of prepwork that needs to be done. Due to the dataset size, only a subset of user ratings will be used - to shorten a computation time.

The following steps will be executesd:

* Load the data :) 
* Select only top 60'000 users - groupbed by ratings count
* Divide users into training - testing - validation groups (randomly)
* Inside a validation group - hide some ratings to simulate real recommendation situation

## Prep from scratch

In [13]:
df = pd.read_csv("./ratings.csv", sep=",", header=0)
# Reset user ids to start from 0 
df.userId = df.userId - 1
df.shape

(20000263, 4)

In [14]:
df.drop('timestamp', axis=1, inplace=True)

In [15]:
df.columns

Index(['userId', 'movieId', 'rating'], dtype='object')

For the purpose of this excercise we will select only top 60k users

In [16]:
user_counts = df.groupby('userId').size().sort_values(ascending=False).reset_index()
user_counts.head(3)

,userId,0
0,118204,9254
1,8404,7515
2,82417,5646


In [14]:
top_users = user_counts.userId.head(60000)

In [15]:
df_top_users = df.loc[df.userId.isin(top_users), :]

Now perform mapping of users to indices and later on - book ISBN numers to indices

In [16]:
df_top_users['uidx'] = df_top_users.userId.astype('category').cat.codes.values
df_top_users['midx'] = df_top_users.movieId.astype('category').cat.codes.values
df_top_users.head(3)

,userId,movieId,rating,uidx,midx
0,0,2,3.5,0,1
1,0,29,3.5,0,28
2,0,32,3.5,0,31


In [13]:
df_top_users.to_csv("./data/processed/movielens/top_users_df.csv", index=False)

Validate results using original dataset

In [17]:
for idx in tqdm(np.random.choice(range(df_top_users.shape[0]), size=100, replace=False)):
    row = df_top_users.iloc[idx, :]
    cnt_by_original_id = df.loc[df.userId == row['userId'], :].shape[0]
    cnt_by_uidx = df_top_users.loc[df_top_users.uidx == row['uidx'], :].shape[0]
    
    assert cnt_by_original_id == cnt_by_uidx

## Train test split by user

In [18]:
unique_users = pd.unique(df_top_users.uidx)
training_users, test_users = train_test_split(unique_users, train_size=0.9, random_state=333)
training_users.shape, test_users.shape

((54000,), (6000,))

In [19]:
train_users, validation_users = train_test_split(training_users, train_size=0.8, random_state=444)
train_users.shape, validation_users.shape

((43200,), (10800,))

In [20]:
train_users_df = df_top_users.loc[df_top_users['uidx'].isin(train_users)]
val_users_df = df_top_users.loc[df_top_users['uidx'].isin(validation_users)]
test_users_df = df_top_users.loc[df_top_users['uidx'].isin(test_users)]

In [21]:
# Reset id values in each group

train_users_df.uidx = train_users_df.uidx.astype('category').cat.codes.values
val_users_df.uidx = val_users_df.uidx.astype('category').cat.codes.values
test_users_df.uidx = test_users_df.uidx.astype('category').cat.codes.values

Validate

In [22]:
for users_df in tqdm([train_users_df, val_users_df, test_users_df]):
    assert(users_df.uidx.min() == 0)
    assert(users_df.midx.min() == 0)

In [213]:
train_users_df.to_csv("./data/processed/movielens/train_users_df.csv", index=False)
test_users_df.to_csv("./data/processed/movielens/test_users_df.csv", index=False)
val_users_df.to_csv("./data/processed/movielens/validation_users_df.csv", index=False)

In [23]:
train_users_df = pd.read_csv("./data/processed/movielens/train_users_df.csv", index_col=False)
test_users_df = pd.read_csv("./data/processed/movielens/test_users_df.csv", index_col=False)
val_users_df = pd.read_csv("./data/processed/movielens/validation_users_df.csv", index_col=False)

# Build full ratings matrices

Recommendation engines based on autoencoders and/or latent factors model use matrices. Therefore we need to construct such:

* Ntr x M - a train matrix of dimensionality: N training users x Movies count
* Ntst x M - a test matrix of dimensionality: N test users x Movies count
* Nval x M - a validation matrix of dimensionality: N validation users x Movies count

Due to their size, those matrices must be sparse (to be able to load into memory)

In [24]:
M = df_top_users.midx.nunique()

In [25]:
train_matrix = lil_matrix((train_users_df.uidx.nunique(), M))
val_matrix = lil_matrix((val_users_df.uidx.nunique(), M))
test_matrix = lil_matrix((test_users_df.uidx.nunique(), M))

In [31]:
pairs = [(train_users_df, train_matrix), (val_users_df, val_matrix), (test_users_df, test_matrix)]

In [99]:
for data, matrix in tqdm(pairs):
    for rowidx in tqdm(range(data.shape[0])):
        row = data.iloc[rowidx, :]
        matrix[int(row['uidx']), int(row['midx'])] = row['rating']

In [100]:
print("done")

done


In [101]:
train_matrix = train_matrix.tocsr()
test_matrix = test_matrix.tocsr()
val_matrix = val_matrix.tocsr()

In [212]:
save_npz(open("./data/processed/movielens/train_matrix.npz", "wb"), train_matrix)
save_npz(open("./data/processed/movielens/test_matrix.npz", "wb"), test_matrix)
save_npz(open("./data/processed/movielens/val_matrix.npz", "wb"), val_matrix)

In [30]:
train_matrix = load_npz(open("./data/processed/movielens/train_matrix.npz", "rb"))
test_matrix = load_npz(open("./data/processed/movielens/test_matrix.npz", "rb"))
val_matrix = load_npz(open("./data/processed/movielens/val_matrix.npz", "rb"))

In [32]:
for data, mat in tqdm(pairs):
    assert(data.rating.sum() == mat.sum())

# Build models

For calculating loss function we take **only observed ratings r**. Therefore we build a *mask* that is pointing only to existing ratings. It will be userd to calculate e.g. reconstruction error and/or quality of ratings.
 

If we took full matrix size for calculating e.g. **MSE** - the value will always be very small (close to 0) due to the number of missing ratings. Therefore - we use only number of existing ratings.

In [33]:
train_mask = (train_matrix > 0.0) * 1.0
test_mask = (test_matrix > 0.0) * 1.0
val_mask = (val_matrix > 0.0) * 1.0

In [34]:
mu = train_matrix.sum() / train_mask.sum()
mu

3.49753038593371

In [35]:
def mape(y_true, y_pred):
    return np.mean(np.abs( (y_true - y_pred)/y_true ))

## Simple autoencoder

In [36]:
reg = 0.0001
lr = 0.08
batch_size = 128
epochs=10

Our simple autoencoder will use special loss function - considering only **existing** ratings. We assume, that latent factors will allow autoencoder to learn real correlations using only ratings that are really present in a dataset.

Given the following:

$$ W_{ih}, W_{ho} \ -\text{weights}  $$
$$ b_{in}, b_{out} \ -\text{ biases } $$
$$ \theta \ - \text{ set of all parametrs for autoencoder} $$
$$ g(x) \ - \text{nonlinear hidden layers activation} $$
$$ f(z) \ - \text{ output activation } $$
$$ \textbf{ r } \ - \text{ set of visible ratings } $$
$$ \textbf{ y } \ - \text{ vector of hidden ratings from test set } $$
$$ h(\textbf{r}; \theta) \ - \text{functional formulation of autoencoder} $$

We can write the following:

$$ h(\textbf{r}; \theta) = f(W_{out} \cdot g(W_{in} \cdot \textbf{r} + b_{in} ) + b_{out}) $$

Out **custom loss function** is defined as:

$$ ℒ(\textbf{r}, h(\textbf{r}; \theta)) = ℒ(\textbf{r}, \hat{\textbf{r}}) = \frac{1}{\left| \textbf{r} \right|} \sqrt{\sum_{i=1}^r \left( r_i - \hat{r_i} \right)} $$ 

In [45]:
def custom_loss(y_true, y_pred):
    mask = tf.cast(tf.not_equal(y_true, 0), dtype='float32')
    y_true = y_true + mu * mask
    y_pred = y_pred + mu * mask
    diff = y_pred - y_true
    sqdiff = diff * diff * mask
    sse = tf.reduce_sum(tf.reduce_sum(sqdiff))
    n = tf.reduce_sum(tf.reduce_sum(mask))
    return sse / n

def custom_perc_loss(y_true, y_pred):
    mask = tf.cast(tf.not_equal(y_true, 0), dtype='float32')
    mape = tf.keras.losses.MeanAbsolutePercentageError()
    return mape(y_true * mask, y_pred * mask)

def generator(ratings, mask, normalize=False):
    while True:
        ratings, mask = shuffle(ratings, mask)
        for i in range(ratings.shape[0] // batch_size + 1):
            upper = min((i+1)*batch_size, ratings.shape[0])
            r = ratings[i*batch_size:upper].toarray()
            m = mask[i*batch_size:upper].toarray()
            if normalize:
                r = r - mu * m
            yield r, r

In [46]:
# input.size -> 1500 -> input.size 

h1 = tf.keras.layers.Dense(
    1500, 
    input_shape=(train_matrix.shape[1], )
    activation='relu', 
    activity_regularizer=tf.keras.regularizers.l1_l2(reg))

d1 = tf.keras.layers.Dropout(rate=0.2)
out = tf.keras.layers.Dense(
    train_matrix.shape[1], 
    kernel_regularizer=tf.keras.regularizers.l2(reg))

model = tf.keras.models.Sequential([h1, d1, out])
model.compile(
  loss=custom_loss,
  optimizer=tf.train.RMSPropOptimizer(learning_rate=lr),
  metrics=[custom_loss, custom_perc_loss])

In [47]:
model_hist = model.fit_generator(
  generator(train_matrix, train_mask),
  validation_data=generator(val_matrix.copy(), val_mask.copy()),
  epochs=epochs,
  steps_per_epoch=train_matrix.shape[0] // batch_size + 1,
  validation_steps=val_matrix.shape[0] // batch_size + 1,
)

Epoch 1/10
338/338 [==============================] - 114s 337ms/step - loss: 59.9457 - custom_loss: 48.9548 - custom_perc_loss: 0.8107 - val_loss: 0.9152 - val_custom_loss: 0.9151 - val_custom_perc_loss: 0.3465
Epoch 2/10
338/338 [==============================] - 112s 332ms/step - loss: 0.8967 - custom_loss: 0.8945 - custom_perc_loss: 0.3393 - val_loss: 0.8883 - val_custom_loss: 0.8882 - val_custom_perc_loss: 0.3420
Epoch 3/10
338/338 [==============================] - 112s 332ms/step - loss: 0.8834 - custom_loss: 0.8834 - custom_perc_loss: 0.3385 - val_loss: 0.8944 - val_custom_loss: 0.8944 - val_custom_perc_loss: 0.3443
Epoch 4/10
338/338 [==============================] - 112s 332ms/step - loss: 0.8804 - custom_loss: 0.8804 - custom_perc_loss: 0.3385 - val_loss: 0.8779 - val_custom_loss: 0.8779 - val_custom_perc_loss: 0.3384
Epoch 5/10
338/338 [==============================] - 115s 339ms/step - loss: 0.8788 - custom_loss: 0.8788 - custom_perc_loss: 0.3383 - val_loss: 0.8849 - val

In [52]:
model.save("./models/movielens_autorec/autorec_simple.h5")

In [47]:
model = tf.keras.models.load_model("./models/movielens_autorec/autorec_simple.h5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [53]:
model.compile(
    loss=custom_loss,
  optimizer=tf.train.RMSPropOptimizer(learning_rate=lr),
  metrics=[custom_loss, custom_perc_loss]
)

### Autorec reconstruction

In [48]:
model.evaluate(test_matrix, test_matrix)

6000/6000 [==============================] - 11s 2ms/sample - loss: 0.8847 - custom_loss: 0.8848 - custom_perc_loss: 0.3325


[0.8847454392115275, 0.8848064, 0.33245188]

In [109]:
model_hist.history.keys()

dict_keys(['custom_perc_loss', 'val_loss', 'val_custom_perc_loss', 'custom_loss', 'val_custom_loss', 'loss'])

In [129]:
plot_loss(model_hist, 'val_custom_perc_loss')

## Multi layer autoencoder

In [73]:
# Model: input.size -> 1500 -> 500 -> 1500 -> input.size

model_autorec_deep = tf.keras.models.Sequential([
    
    tf.keras.layers.Dense(1500, activation='relu', activity_regularizer=tf.keras.regularizers.l1_l2(reg), input_shape=(train_matrix.shape[1], )),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(500, activation='relu', activity_regularizer=tf.keras.regularizers.l1_l2(reg)),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(1500, activation='relu', activity_regularizer=tf.keras.regularizers.l1_l2(reg)),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(train_matrix.shape[1], kernel_regularizer=tf.keras.regularizers.l1_l2(reg))
])

model_autorec_deep.compile(
  loss=custom_loss,
  optimizer=tf.train.AdamOptimizer(lr),
  metrics=[custom_loss, custom_perc_loss],
)

In [74]:
hist_deep_autorec = model_autorec_deep.fit_generator(
  generator(train_matrix, train_mask, normalize=False),
  validation_data=generator(val_matrix.copy(), val_mask.copy()),
  epochs=5,
  steps_per_epoch=train_matrix.shape[0] // batch_size + 1,
  validation_steps=val_matrix.shape[0] // batch_size + 1,
)

Epoch 1/5
338/338 [==============================] - 133s 395ms/step - loss: 679.3275 - custom_loss: 184.6780 - custom_perc_loss: 0.7801 - val_loss: 89.1296 - val_custom_loss: 1.1699 - val_custom_perc_loss: 0.3467
Epoch 2/5
338/338 [==============================] - 130s 384ms/step - loss: 112.1540 - custom_loss: 6.8580 - custom_perc_loss: 0.3552 - val_loss: 72.1368 - val_custom_loss: 0.9466 - val_custom_perc_loss: 0.3416
Epoch 3/5
338/338 [==============================] - 130s 385ms/step - loss: 113.9213 - custom_loss: 2.6676 - custom_perc_loss: 0.3379 - val_loss: 78.5351 - val_custom_loss: 0.8785 - val_custom_perc_loss: 0.3462
Epoch 4/5
338/338 [==============================] - 130s 385ms/step - loss: 83.9787 - custom_loss: 1.7218 - custom_perc_loss: 0.3375 - val_loss: 52.1888 - val_custom_loss: 0.8795 - val_custom_perc_loss: 0.3414
Epoch 5/5
338/338 [==============================] - 130s 386ms/step - loss: 72.7003 - custom_loss: 1.7114 - custom_perc_loss: 0.3372 - val_loss: 65.15

In [75]:
model_autorec_deep.evaluate(test_matrix, test_matrix)

6000/6000 [==============================] - 15s 3ms/sample - loss: 65.1643 - custom_loss: 0.8750 - custom_perc_loss: 0.3292


[65.1642725423177, 0.87495154, 0.32919633]

In [76]:
hist_deep_autorec.history.keys()

dict_keys(['custom_perc_loss', 'val_loss', 'val_custom_perc_loss', 'custom_loss', 'val_custom_loss', 'loss'])

In [130]:
plot_loss(hist_deep_autorec, 'custom_perc_loss')

### Autorec missing ratings pred

In [132]:
test_matrix.shape

(6000, 26672)

In [96]:
test_ratings_nonzero_u, test_ratings_nonzero_m  = test_matrix.nonzero()
test_ratings_nonzero = list(zip(test_ratings_nonzero_u, test_ratings_nonzero_m))

In [97]:
test_ratings_nonzero[:3]

[(0, 184), (0, 257), (0, 293)]

In [98]:
test_ratings_to_hide = shuffle(test_ratings_nonzero, random_state=666)[:2000]

In [99]:
test_matrix_partial = test_matrix.copy()
expected = []
for u, m in tqdm(test_ratings_to_hide):
    val = test_matrix_partial[u,m]
    expected.append(val)
    test_matrix_partial[u,m] = 0.0
    
expected = np.array(expected)

In [105]:
yhat_partial = model_autorec_deep.predict(test_matrix_partial).clip(min=0.0, max=5.0)

In [106]:
mean_squared_error(test_matrix_partial.todense(), yhat_partial)

9.66246184468125

In [107]:
actual = []
for u, m in tqdm(test_ratings_to_hide):
    actual.append(yhat_partial[u, m])
actual = np.array(actual)

Final prediction error for **selected (hidden) ratings** from test matrix.

In [108]:
np.mean(np.abs(  (expected-actual)/expected  ))

0.3124175567379074

## Latent factors

Below we build a latent factors model, which will try to accomplish the same task - matrix reconstruction and rating prediction, but using different model.

We create a huge embedding matrices
* $ U x F\ \text{ matrix of users-to-latent factors } $
* $ M x F\ \text{ matrix of movies-to-latent factors}  $

And then minimize reconstruction error by multiplying those weights using dot product.

In [76]:
train_top_users = df_top_users.loc[df_top_users.uidx.isin(train_users), :]
val_top_users = df_top_users.loc[df_top_users.uidx.isin(validation_users), :]
test_top_users = df_top_users.loc[df_top_users.uidx.isin(test_users), :]

In [78]:
M = df_top_users.midx.nunique()

In [79]:
n_latent_factors = 15

In [80]:
u = tf.keras.layers.Input(shape=(1,))
m = tf.keras.layers.Input(shape=(1,))
u_embedding = tf.keras.layers.Embedding(df_top_users.uidx.nunique(), n_latent_factors, embeddings_regularizer=tf.keras.regularizers.l2(reg))(u) # (N, 1, K)
m_embedding = tf.keras.layers.Embedding(M, n_latent_factors, embeddings_regularizer=tf.keras.regularizers.l2(reg))(m) # (N, 1, K)

u_bias = tf.keras.layers.Embedding(df_top_users.uidx.nunique(), 1, embeddings_regularizer=tf.keras.regularizers.l2(reg))(u) # (N, 1, 1)
m_bias = tf.keras.layers.Embedding(M, 1, embeddings_regularizer=tf.keras.regularizers.l2(reg))(m) # (N, 1, 1)
x = tf.keras.layers.Dot(axes=2)([u_embedding, m_embedding]) # (N, 1, 1)
x = tf.keras.layers.Add()([x, u_bias, m_bias])
x = tf.keras.layers.Flatten()(x) # (N, 1)

model_mf = tf.keras.Model(inputs=[u, m], outputs=x)

In [81]:
model_mf.compile(
  loss='mse',
  optimizer=tf.train.RMSPropOptimizer(learning_rate=0.08),
  metrics=['mse'],
)

In [ ]:
r = model_mf.fit(
  x=[train_top_users.uidx, train_top_users.midx],
  y=train_top_users.rating,
  epochs=5,
  batch_size=128,
  validation_data=(
    [val_top_users.uidx, val_top_users.midx],
    val_top_users.rating
  )
)

Train on 12054920 samples, validate on 3015043 samples
Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


Epoch 1/5
   36096/12054920 [..............................] - ETA: 39:12 - loss: 9.0017 - mean_squared_error: 8.6692

In [84]:
print("done")

done


In [ ]:
iplot([Scatter({'y': r.history['loss']})])

In [98]:
yhat_mf = model_mf.predict([test_users_df.uidx, test_users_df.midx])

In [99]:
yhat_mf = yhat_mf.clip(min=0, max=5.0)

In [100]:
actual_mf = []
for idx in tqdm(range(test_users_df.shape[0])):
    row = test_users_df.iloc[idx,:]
    actual_mf.append(yhat_mf[int(row['uidx'])])
actual_mf = np.array(actual_mf)

In [101]:
np.mean(np.abs(  (test_users_df.rating - actual_mf[:, 0])/test_users_df.rating  ))

0.6129794534856234